Import necessary packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500)

Import datasets and packages

In [21]:
# df_lookup = pd.read_csv('../../data/raw/EXTR_LookUp.csv', dtype='str')
df_parcel = pd.read_csv('../../data/raw/EXTR_PARCEL.csv', dtype='str')
df_resbldg = pd.read_csv('../../data/raw/EXTR_ResBldg.csv', dtype='str')
df_rpsale = pd.read_csv('../../data/raw/EXTR_RPSale.csv', dtype='str')

In [3]:
df_resbldg

,Major,Minor,BldgNbr,NbrLivingUnits,Address,BuildingNumber,Fraction,DirectionPrefix,StreetName,StreetType,...,FpMultiStory,FpFreestanding,FpAdditional,YrBuilt,YrRenovated,PcntComplete,Obsolescence,PcntNetCondition,Condition,AddnlCost
0,009800,0720,1,1,27719 SE 26TH WAY 98075,27719,,SE,26TH,WAY,...,0,0,0,2001,0,0,0,0,3,0
1,009802,0140,1,1,2829 277TH TER SE 98075,2829,,,277TH,TER,...,0,0,0,2004,0,0,0,0,3,0
2,009830,0020,1,1,1715 298TH CRESENT SE,1715,,,298TH CRESENT,,...,0,0,0,2017,0,0,0,0,3,0
3,009830,0160,1,1,1861 297TH WAY SE 98024,1861,,,297TH,WAY,...,0,0,0,2013,0,0,0,0,3,0
4,010050,0180,1,1,35410 25TH PL S 98003,35410,,,25TH,PL,...,0,0,0,1994,0,0,0,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181505,197220,1902,1,1,3816 WINSLOW PL N 98103,3816,,,WINSLOW,PL,...,0,0,0,2003,0,0,0,0,3,0
181506,197220,2224,1,1,3841 B WOODLAND PARK AVE N 98103,3841,B,,WOODLAND PARK,AVE,...,0,0,0,2005,0,0,0,0,3,0
181507,197220,5172,1,1,11 B W DRAVUS ST,11,B,W,DRAVUS,ST,...,0,0,0,2007,0,0,0,0,3,0
181508,197220,5173,1,1,11 C W DRAVUS ST,11,C,W,DRAVUS,ST,...,0,0,0,2007,0,0,0,0,3,0


In [4]:
df_resbldg.columns

Index(['Major', 'Minor', 'BldgNbr', 'NbrLivingUnits', 'Address',
       'BuildingNumber', 'Fraction', 'DirectionPrefix', 'StreetName',
       'StreetType', 'DirectionSuffix', 'ZipCode', 'Stories', 'BldgGrade',
       'BldgGradeVar', 'SqFt1stFloor', 'SqFtHalfFloor', 'SqFt2ndFloor',
       'SqFtUpperFloor', 'SqFtUnfinFull', 'SqFtUnfinHalf', 'SqFtTotLiving',
       'SqFtTotBasement', 'SqFtFinBasement', 'FinBasementGrade',
       'SqFtGarageBasement', 'SqFtGarageAttached', 'DaylightBasement',
       'SqFtOpenPorch', 'SqFtEnclosedPorch', 'SqFtDeck', 'HeatSystem',
       'HeatSource', 'BrickStone', 'ViewUtilization', 'Bedrooms',
       'BathHalfCount', 'Bath3qtrCount', 'BathFullCount', 'FpSingleStory',
       'FpMultiStory', 'FpFreestanding', 'FpAdditional', 'YrBuilt',
       'YrRenovated', 'PcntComplete', 'Obsolescence', 'PcntNetCondition',
       'Condition', 'AddnlCost'],
      dtype='object')

In [5]:
price = df_rpsale.SalePrice

# plt.boxplot(price);

In [6]:
def remove_extremes(data, devct):
    data = pd.Series([float(num) for num in data])
    cleaned = data.loc[data>0].copy()

    std = cleaned.std()
    med = cleaned.median()

    cleaned = cleaned.loc[(cleaned > (med - std*devct)) & (cleaned < (med+std*devct))].copy() 
    return cleaned

In [7]:
price = remove_extremes(df_rpsale.SalePrice, 3)
#price.hist();
#plt.boxplot(price);

In [8]:
df_rpsale.columns

Index(['ExciseTaxNbr', 'Major', 'Minor', 'DocumentDate', 'SalePrice',
       'RecordingNbr', 'Volume', 'Page', 'PlatNbr', 'PlatType', 'PlatLot',
       'PlatBlock', 'SellerName', 'BuyerName', 'PropertyType', 'PrincipalUse',
       'SaleInstrument', 'AFForestLand', 'AFCurrentUseLand', 'AFNonProfitUse',
       'AFHistoricProperty', 'SaleReason', 'PropertyClass', 'SaleWarning'],
      dtype='object')

In [9]:
maj_min = pd.DataFrame([df_rpsale.Major, df_rpsale.Minor]).T

In [10]:
maj_min['Major-Minor'] = maj_min.Major+'-'+maj_min.Minor
maj_min['Major-Minor'].value_counts().sort_values(ascending=False).head(100)

066000-2310    83
262505-9212    39
884750-0004    38
042504-9042    30
342506-9018    23
232204-9082    23
775780-0220    22
329370-0095    21
342105-9008    20
066000-2325    20
510140-0770    19
539665-0004    19
332605-9152    17
011410-0979    17
927015-0009    17
202406-9115    16
606125-0080    16
327840-0004    14
177760-0060    14
437795-0003    14
222530-0144    13
866487-0010    13
866487-0120    13
142305-9023    13
072303-9070    13
214480-0500    13
762570-3235    13
192430-0004    13
261910-0014    13
532310-0141    12
617290-0230    12
342405-9029    12
042406-9159    12
732755-0670    11
301930-0070    11
524780-0030    11
264370-0020    11
785355-0004    11
143240-0800    11
788360-4085    11
082211-9001    10
042406-9174    10
884390-0445    10
788360-0665    10
516970-0075    10
360300-0160    10
152203-9065    10
388690-2240     9
232306-9021     9
200200-0020     9
124070-0070     9
335340-1475     9
012980-0240     9
952810-1345     9
137080-1275     9
394440-005

**Investigating Residential Building "ResBldg" file below**

In [11]:
df_resbldg = pd.read_csv('../../data/raw/EXTR_ResBldg.csv', dtype='str')
df_resbldg.head(3)

,Major,Minor,BldgNbr,NbrLivingUnits,Address,BuildingNumber,Fraction,DirectionPrefix,StreetName,StreetType,...,FpMultiStory,FpFreestanding,FpAdditional,YrBuilt,YrRenovated,PcntComplete,Obsolescence,PcntNetCondition,Condition,AddnlCost
0,009800,0720,1,1,27719 SE 26TH WAY 98075,27719,,SE,26TH,WAY,...,0,0,0,2001,0,0,0,0,3,0
1,009802,0140,1,1,2829 277TH TER SE 98075,2829,,,277TH,TER,...,0,0,0,2004,0,0,0,0,3,0
2,009830,0020,1,1,1715 298TH CRESENT SE,1715,,,298TH CRESENT,,...,0,0,0,2017,0,0,0,0,3,0


In [12]:
df_resbldg.columns

Index(['Major', 'Minor', 'BldgNbr', 'NbrLivingUnits', 'Address',
       'BuildingNumber', 'Fraction', 'DirectionPrefix', 'StreetName',
       'StreetType', 'DirectionSuffix', 'ZipCode', 'Stories', 'BldgGrade',
       'BldgGradeVar', 'SqFt1stFloor', 'SqFtHalfFloor', 'SqFt2ndFloor',
       'SqFtUpperFloor', 'SqFtUnfinFull', 'SqFtUnfinHalf', 'SqFtTotLiving',
       'SqFtTotBasement', 'SqFtFinBasement', 'FinBasementGrade',
       'SqFtGarageBasement', 'SqFtGarageAttached', 'DaylightBasement',
       'SqFtOpenPorch', 'SqFtEnclosedPorch', 'SqFtDeck', 'HeatSystem',
       'HeatSource', 'BrickStone', 'ViewUtilization', 'Bedrooms',
       'BathHalfCount', 'Bath3qtrCount', 'BathFullCount', 'FpSingleStory',
       'FpMultiStory', 'FpFreestanding', 'FpAdditional', 'YrBuilt',
       'YrRenovated', 'PcntComplete', 'Obsolescence', 'PcntNetCondition',
       'Condition', 'AddnlCost'],
      dtype='object')

**ADD PORCH & HEATING METRICS TO RESBLDG DATAFRAME**

In [66]:
resbldg = df_resbldg[['SqFtOpenPorch', 'SqFtEnclosedPorch']].copy()
resbldg['Parcel_ID'] = df_resbldg['Major'] + '-' + df_resbldg['Minor']
resbldg = resbldg[['Parcel_ID', 'SqFtOpenPorch', 'SqFtEnclosedPorch']].copy()

resbldg['SqFtOpenPorch'] = resbldg['SqFtOpenPorch'].astype('int')
resbldg['SqFtEnclosedPorch'] = resbldg['SqFtEnclosedPorch'].astype('int')
resbldg['HeatSystem'] = df_resbldg['HeatSystem']
resbldg

,Parcel_ID,SqFtOpenPorch,SqFtEnclosedPorch,HeatSystem
0,009800-0720,0,0,5
1,009802-0140,380,0,5
2,009830-0020,360,0,5
3,009830-0160,690,0,5
4,010050-0180,60,0,5
...,...,...,...,...
181505,197220-1902,0,0,5
181506,197220-2224,0,0,5
181507,197220-5172,0,0,5
181508,197220-5173,0,0,5


**ONE-HOT ENCODE HEAT SYSTEM METRICS**

In [64]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()

,Parcel_ID,SqFtOpenPorch,SqFtEnclosedPorch,HeatSystem
0,009800-0720,0,0,5
1,009802-0140,380,0,5
2,009830-0020,360,0,5
3,009830-0160,690,0,5
4,010050-0180,60,0,5
...,...,...,...,...
181505,197220-1902,0,0,5
181506,197220-2224,0,0,5
181507,197220-5172,0,0,5
181508,197220-5173,0,0,5


In [63]:



# ohe.fit(listparcel['StreetSurface'])

In [62]:
df_resbldg.columns

Index(['Major', 'Minor', 'BldgNbr', 'NbrLivingUnits', 'Address',
       'BuildingNumber', 'Fraction', 'DirectionPrefix', 'StreetName',
       'StreetType', 'DirectionSuffix', 'ZipCode', 'Stories', 'BldgGrade',
       'BldgGradeVar', 'SqFt1stFloor', 'SqFtHalfFloor', 'SqFt2ndFloor',
       'SqFtUpperFloor', 'SqFtUnfinFull', 'SqFtUnfinHalf', 'SqFtTotLiving',
       'SqFtTotBasement', 'SqFtFinBasement', 'FinBasementGrade',
       'SqFtGarageBasement', 'SqFtGarageAttached', 'DaylightBasement',
       'SqFtOpenPorch', 'SqFtEnclosedPorch', 'SqFtDeck', 'HeatSystem',
       'HeatSource', 'BrickStone', 'ViewUtilization', 'Bedrooms',
       'BathHalfCount', 'Bath3qtrCount', 'BathFullCount', 'FpSingleStory',
       'FpMultiStory', 'FpFreestanding', 'FpAdditional', 'YrBuilt',
       'YrRenovated', 'PcntComplete', 'Obsolescence', 'PcntNetCondition',
       'Condition', 'AddnlCost'],
      dtype='object')